## Import data

In [1]:
# Import package
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# Fix view Chinese problem 
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['Microsoft YaHei']    # 指定預設字型:解決plot不能顯示中文問題
mpl.rcParams['axes.unicode_minus'] = False      

In [3]:
# print current working directory
print(os.getcwd())

C:\Users\User\Desktop\2020_DataPilot\Code


In [5]:
# Change working directory
os.chdir('C:\\Users\\User\\Desktop\\2020_DataPilot\\Data\\PChome_data') 
print(os.getcwd())

C:\Users\User\Desktop\2020_DataPilot\Data\PChome_data


In [6]:
#Import data
PChome = pd.read_csv('Full_PChome_utf.csv', encoding = 'utf-8')
PChome = PChome.drop(['idx','postal_cd', 'date_cd', 'goods','Zip_Code'], axis = 1)
PChome['date_create'] = pd.to_datetime(PChome['date_create'])
PChome['year'] = PChome['date_create'].map(lambda x: x.year)
PChome['month'] = PChome['date_create'].map(lambda x: x.month)
PChome['day'] = PChome['date_create'].map(lambda x: x.day)
PChome.head()

,member_id,prod_id,department,index,prod_Q,price,MSRP,date_create,P_Q,Region,goods_clean,brand,year,month,day
0,15416142,QAAD70-A9006TWM5-000,3C,列 印,1,6490,NaN,2019-10-09,6490,高雄市,HP Officejet 7110 A3 網路高速印表機,HP,2019,10,9
1,4663689,QFAXAS-A900A38S1-001,生活,衛浴,1,599,NaN,2019-10-09,599,嘉義市,HappyLife 桌上型微波爐架 烤箱置物架 YV9799,HappyLife,2019,10,9
2,8097088,QFAAJY-A9009W99R-000,生活,傢俱,1,927,1359.0,2019-10-09,927,高雄市,dayneeds 輕型 45 45 160cm 四層電鍍波浪收納鐵架,dayneeds,2019,10,9
3,15317712,QAAS33-A900A7SR4-002,3C,電 玩,1,690,990.0,2019-10-09,690,臺南市,任天堂 Switch Joy Con 四合一鋁合金 手把充電座兩色可選,任天堂,2019,10,9
4,15438512,CGAACT-A80423382-005,運動戶外,汽百/記錄器,1,1055,1600.0,2019-10-09,1055,臺南市,車用攝影鏡頭,車用攝影鏡頭,2019,10,9


In [7]:
# Replace all
def replace_all(text, dic):
    for key in dic:
        text = text.replace(key, d[key])
    return text

In [38]:
# Preprocess Apriori data
d = {" ": "", "\u3000": "", ".":"/", "?":""}
Apriori = pd.DataFrame()
Apriori['member_id'] = PChome['member_id']
Apriori['date'] = PChome['date_create']
Apriori['goods'] = PChome['index'].map(lambda x: replace_all(x, d))
Apriori['goods'] = Apriori.groupby(['member_id', 'date'])['goods'].transform(lambda x: ','.join(x))
Apriori = Apriori[['member_id', 'date', 'goods']].drop_duplicates()
Apriori.head()

,member_id,date,goods
0,15416142,2019-10-09,列印
1,4663689,2019-10-09,衛浴
2,8097088,2019-10-09,"傢俱,書房"
3,15317712,2019-10-09,電玩
4,15438512,2019-10-09,汽百/記錄器


In [39]:
# To_csv
Apriori.to_csv('Apriori.csv')

## Apriori

In [40]:
# Preprocess 
transactions = []
dataset = Apriori['goods'].str.split(',', expand = True)
for i in range(dataset.shape[0]):
    transactions.append([str(dataset.values[i,j]) for j in range(dataset.shape[1]) if str(dataset.values[i,j]) != 'None'])

In [41]:
# apriori package
from apyori import apriori
rules = apriori(transactions = transactions, min_support = 0.00001,min_confidence = 0.0001)

In [236]:
# inspect
def inspect(result):
    resultsinDataFrame = []
    for i in range(len(result)):
        if (len(result[i][0]) == 2) & (len(results[i][2]) == 2):
            resultsinDataFrame.append({
                'lhs_goods' : str(result[i][0]).split(',')[0].split("'")[1],
                'rhs_goods' : str(results[i][0]).split(',')[1].split("'")[1],
                'supports' : round(results[i][1],5),
                'l_confidence' : float(str(result[i][2]).split(',')[2].split('=')[1]),
                'r_confidence' : float(str(result[i][2]).split(',')[6].split('=')[1]),
                'l_lift' : float(str(results[i][2]).split(',')[3].split('=')[1].split(')')[0]),
                'r_lift' : float(str(results[i][2]).split(',')[7].split('=')[1].split(')')[0])
            }) 
    return pd.DataFrame(resultsinDataFrame, columns = \
                                  ['lhs_goods', 'rhs_goods', 'supports', 'l_confidence', 'r_confidence', 'l_lift', 'r_lift'])

In [240]:
# To_csv
inspect(results).to_csv('Apriori_result.csv', encoding = 'big5')